In [1]:
import sys
from pathlib import Path

entry_path =  str(Path().resolve().parents[0])
entry_path

'/home/exodia/NAO'

In [2]:
!pip install optuna optunahub cmaes scipy

In [4]:
import sys
import optuna
import optunahub
from pathlib import Path
import torch
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"



sys.path.append(entry_path)


from scripts.train.train_cnn import main  
import matplotlib.pyplot as plt
import itertools
import pandas as pd

param_grid = {
    'model': ['mobilenet', 'vgg19'],
    'batch_size': [16, 32],
    'lr': [1e-3, 1e-4],
    'epochs': [10],
}

model = 'mobilenet'

data_path = entry_path+'/data/kers2013_sample_500_val20/' 


In [ ]:
# batch_size, epochs, lr = 4, 1, 0.3
# val_acc = main(model, batch_size, epochs, lr, data_path)


In [6]:
from tqdm import tqdm
import optuna
from optuna.pruners import MedianPruner
import gc

n_trials = 10
epochs = 10
models = ['mobilenet', 'vgg19', 'custom']
data_path = entry_path + '/data/kers2013/'

for model in models:
    print(f"\n🔍 Optimizando para modelo: {model}")

    # 🎯 Función objetivo con soporte para pruning
    def objective(trial: optuna.Trial) -> float:
        torch.cuda.empty_cache()
        gc.collect()
        if model == 'vgg19':
            batch_size_options = [16, 32]
        else:
            batch_size_options = [32, 48, 64]
        batch_size = trial.suggest_categorical("batch_size", batch_size_options)

        lr = trial.suggest_float("lr", 1e-4, 1e-3)

        try:
            history = main(model, batch_size, epochs, lr, data_path, trial=trial)

            for epoch, val_acc in enumerate(history['val_accuracy']):
                trial.report(val_acc, step=epoch)
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
                
            trial.set_user_attr("model", history["model"])

        except RuntimeError as e:
            if "out of memory" in str(e):
                print("⚠️ Trial falló por falta de memoria")
                torch.cuda.empty_cache()
                raise optuna.exceptions.TrialPruned()
            else:
                raise


        return history['val_accuracy'][-1]

    # 🔧 Carga módulo + pruning
    module = optunahub.load_module(package="samplers/auto_sampler")
    pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=3)
    study = optuna.create_study(sampler=module.AutoSampler(), pruner=pruner, direction="maximize")

    # 📊 tqdm callback
    pbar = tqdm(total=n_trials, desc=f"{model}")
    def tqdm_callback(study, trial):
        pbar.update(1)

    # 🚀 Optimización
    study.optimize(objective, n_trials=n_trials, callbacks=[tqdm_callback])
    pbar.close()


    if len(study.trials) > 0 and study.best_trial.user_attrs.get("model"):
        best_model = study.best_trial.user_attrs["model"]
        trial_num = study.best_trial.number
        best_model_path = f"{data_path}../models/{model}_trial{trial_num}_best.pt"
        torch.save(best_model.state_dict(), best_model_path)
        print(f"✅ Mejor modelo guardado en: {best_model_path}")
    else:
        print(f"⚠️ No se pudo guardar modelo para {model}. Todos los trials fallaron o fueron podados.")




[I 2025-06-21 10:59:01,050] A new study created in memory with name: no-name-e339ed91-0e24-4f5f-a693-eb381000999d



🔍 Optimizando para modelo: mobilenet


mobilenet:   0%|          | 0/10 [00:00<?, ?it/s]2025-06-21 10:59:01,153 - 📄 Trial parameters saved to /home/exodia/NAO/data/kers2013/../../notebooks/logs/mobilenet_trial0/20250621_105901_params.txt
2025-06-21 10:59:01,154 - 📁 Training log created for model: mobilenet_trial0
2025-06-21 10:59:01,154 - 📄 Logs will be saved to /home/exodia/NAO/data/kers2013/../../notebooks/logs/mobilenet_trial0/20250621_105901.log
2025-06-21 10:59:01,154 - 📄 CSV will be saved to /home/exodia/NAO/data/kers2013/../../notebooks/logs/mobilenet_trial0/20250621_105901.csv
/home/exodia/miniconda3/envs/nao_sv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/exodia/miniconda3/envs/nao_sv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may

Epoch 1/10 | Train Loss: 1.1602 | Val Loss: 1.0173 | Val Acc: 0.6158 | Time: 125.76s


2025-06-21 11:02:54,373 - Epoch 2/10 | Train Loss: 0.9468 | Val Loss: 0.9751 | Val Acc: 0.6364 | Time: 107.22s


...

2025-06-21 18:51:26,181 - Epoch 9/10 | Train Loss: 1.0753 | Val Loss: 1.2636 | Val Acc: 0.5222 | Time: 6.19s


Epoch 9/10 | Train Loss: 1.0753 | Val Loss: 1.2636 | Val Acc: 0.5222 | Time: 6.19s


2025-06-21 18:51:32,383 - Epoch 10/10 | Train Loss: 1.0259 | Val Loss: 1.2854 | Val Acc: 0.5213 | Time: 6.20s
2025-06-21 18:51:32,384 - Trial podado.
2025-06-21 18:51:32,384 - ✅ Logging finalizado.
[W 2025-06-21 18:51:32,385] Trial 5 failed with parameters: {'batch_size': 64, 'lr': 0.00021157245088287074} because of the following error: NameError("name 'optuna' is not defined").
Traceback (most recent call last):
  File "/home/exodia/miniconda3/envs/nao_sv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_13043/4076753600.py", line 27, in objective
    history = main(model, batch_size, epochs, lr, data_path, trial=trial)
  File "/home/exodia/NAO/scripts/train/train_cnn.py", line 110, in main
    raise optuna.exceptions.TrialPruned()
NameError: name 'optuna' is not defined
[W 2025-06-21 18:51:32,386] Trial 5 failed with value None.


Epoch 10/10 | Train Loss: 1.0259 | Val Loss: 1.2854 | Val Acc: 0.5213 | Time: 6.20s


NameError: name 'optuna' is not defined